<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/1-Introduction%20to%20LLMs%20with%20OpenAI/1_1-First_Chatbot_OpenAI_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<h1>Large Language Models Projects</h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>1.1-Create your first Chatbot With OpenAI and GRADIO</h2>
</div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra/)
___________

Models: gpt-3.5-turbo / gpt-4o-mini

Colab Environment: CPU

Keys:
* OpenAI roles.
* Memory in conversations.
* Gradio
___________________________



# Vertical Chat Gradio
How to build a chat for small businees using:

* GPT 3.5 / GPT-4o mini
* Gradio
* OpenAI


This is just a simple sample to start understanding how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution, but we are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?
* How Gradio works.

It is based in the first example of the course that use Panel instead of Gradio.
https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/1-Introduction%20to%20LLMs%20with%20OpenAI/1_1-First_Chatbot_OpenAI.ipynb


In [1]:
#First install the necesary libraries
!pip install -q openai==1.1.1
!pip install -q gradio==4.40.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130

In [2]:
#if you need a API Key from OpenAI
#https://platform.openai.com/account/api-keys
import openai
import random
import gradio as gr

from getpass import getpass
openai.api_key=getpass("OpenAI API Key: ")
#model = "gpt-3.5-turbo"
model = "gpt-4o-mini"

OpenAI API Key: ··········


At OpenAI, we find three distinct roles:

* system: Provides instructions to the model on how it should behave. We can define its personality here.
* user: This role is used to send messages from the user.
* assistant: The responses generated by the model.

In the *context* variable, we will store the instructions for the model, which contain how it should act and the ice cream shop's menu.

In [3]:
#Creating the system part of the prompt
#Read and understand it.

context = [ {'role':'system', 'content':"""
You work collecting orders in a delivery IceCream shop called
I'm freezed.

First welcome the customer, in a very friedly way, then collects the order.

Your instuctions are:
-Collect the entire order, only from options in our menu, toppings included.
-Summarize it
-check for a final time if everithing is ok or the customer wants to add anything else.
-collect the payment, be sure to include topings and the size of the ice cream.
-Make sure to clarify all options, extras and sizes to uniquely
identify the item from the menu.
-Your answer should be short in a very friendly style.

Our Menu:
The IceCream menu includes only the flavors:
-Vainilla.
-Chocolate.
-Lemon.
-Strawberry.
-Coffe.

The IceCreams are available in two sizes:
-Big: 3$
-Medium: 2$

Toppings:
-Caramel
-White chocolate
-melted peanut butter
Each topping cost 0.5$

"""} ]

This function is just a wrapper for the OpenAI API. It will receives the messages in OpenAI format and return the model response.

Sample of OpenAI Conversation:

[{"role":"user", "content":"hi"},
 {"role":"system", "content":"what can I do for you."}]

In [4]:
#This function will receive the different messages in the conversation,
#and call OpenAI passing the full conversartion.
def continue_conversation(messages, temperature=0):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content


The GradioChat function will be called by Gradio and is responsible for maintaining the conversation. In the first parameter, it will receive the current message from the user, and in the second parameter, the complete history with all previous messages and responses.

In this case, Gradio is responsible for maintaining the conversation history. This function is responsible for formatting it to work correctly with OpenAI models.

In [6]:
#The function that Gradio will use.
def gradio_chat(message, history):
    #Add the instructions to the prompt.
    history_chat = context

    #Add the history that Gradio send to us.
    for user, assistant in history:
        history_chat.append({"role":"user", "content":user})
        history_chat.append({"role":"assistant", "content":assistant})

    #Add the las user message.
    history_chat.append({"role":"user", "content":message})

    #Call OpenAI and return the response.
    return continue_conversation(history_chat, 0)

In [7]:
#Customized gradio textbox.
InputText = gr.Textbox(label="order", info="Your Order here.", scale= 6)

I'm going to use the simplest Gradio function for creating chats: *ChatInterface*. The first parameter it should receive is the previously built function responsible for maintaining the dialogue with OpenAI. In this case, *gradio_chat*.

The function can be parameterized with a multitude of parameters, you can consult them in the official Gradio documentation: https://www.gradio.app/docs/gradio/chatinterface

In this case, I pass it a customized textbox, otherwise we would have the default one, which would work the same, but without the customized text. I indicate that it should not show the retry or undo buttons, in addition to giving it a title and changing the text of the Submit button.



In [8]:
gr.ChatInterface(gradio_chat,
                 textbox=InputText,
                 retry_btn=None,
                 undo_btn=None,
                 title="I'm freezed",
                submit_btn="Order").launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d45c3e8b6896f165b7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Gradio is very simple to use, yet incredibly powerful in terms of customization. It's a great tool for creating demos and quickly seeing how the built solutions work.